In [1]:
import random

# Modules to import

from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.vectorstores.faiss import FAISS

import os
from dotenv import load_dotenv, find_dotenv

# Get OpenAI API KEY

In [2]:
load_dotenv(find_dotenv()) # read local .env file
api = os.getenv('OPENAI_API_KEY_H')

# Split Chunk

In [3]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=600,
    chunk_overlap=100,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""]
)

docs = []
for loader in os.listdir("docs"):
    if loader is not None :
        try :
            docs.extend(PyPDFLoader("docs/"+loader).load())
        except Exception as e1 :
            print(f" [ Documents ERROR ] : {e1}")
    else : break

In [4]:
docs[:2]

[Document(page_content='1-72 สารบัญ ภาพโดยรวม/คู่มืออย่างง่าย        1 ข้อมูลทั่วไป          2 การล็อกและการปลดล็อก        3 เบาะนั่งและเข็มขัดนิรภัย         4 แผงหน้าป้ดและอุปกรณ์ควบคุม        5 การสตาร์ทและการขับขี่         6 การใช้อุปกรณ์อํานวยความสะดวก       7 เมื่อเกิดเหตุฉุกเฉิน         8 การดูแลรักษาสภาพรถ         9 การบํารุงรักษา          10 ข้อมูลจําเพาะ          11 ', metadata={'source': 'docs/Text_PDF_.pdf', 'page': 0}),
 Document(page_content='แผงหน้าป้ดและอุปกรณ์ควบคุม(ด้านคนขับ) ภาพโดยรวม/คู่มืออย่างง่าย1-11 - สวิตช์ที่ป้ดนํ้าฝนและฉีดนํ้าล้างกระจกหน้า5-45 - แผงหน้าป้ดหน้า5-2 - สวิตช์ควบคุมความสว่างของแผงหน้าป้ดหน้า5-2 - สวิตช์ระบบควบคุมเสถียรภาพการทรงตัว(ASC)OFFหน้า6-70 - สวิตช์เซ็นทรัลล็อกหน้า3-15 - สวิตช์ควบคุมหน้าต่างไฟฟ้าหน้า3-19 - สวิตช์ล็อกหน้า3-20 - สวิตช์ไฟหน้าและไฟสูง-ตํ่าหน้า5-40 - สวิตช์ระบบปรับไฟสูงอัตโนมัติ(AHB)หน้า5-43 - คันสวิตช์ไฟเลี้ยวหน้า5-44 - สวิตช์ไฟตัดหมอกด้านหน้าหน้า5-45 - สวิตช์หน้าจอแสดงข้อมูลรวมหน้า5-4 - สวิตช์ควบคุมเครื่องเสียงบนพวงมาลัยให้ดูคู่

# Vector Database

In [5]:
splits = r_splitter.split_documents(docs)

db = FAISS.from_documents( splits, OpenAIEmbeddings( api_key = api ) )

In [6]:
splits[random.randint(0,len(splits))]

Document(page_content='ถุงลมนิรภัยจะทํางานก็ต่อเมื่อสวิตช์กุญแจหรือโหมดการทํางานอยู่ภายใต้เงื่อนไขต่อไปนี้[ยกเว้นรถรุ่นที่ติดตั้งระบบกุญแจอัจฉริยะ]สวิตช์กุญแจอยู่ในตําแหน่ง “ON” หรือ “START”[รถรุ่นที่ติดตั้งระบบกุญแจอัจฉริยะ]โหมดการทํางานอยู่ที่ ONถุงลมนิรภัยจะพองตัวอย่างรวดเร็วพร้อมๆ กับเกิดเสียงดังและมีฝุ๋นควันคลุ้งกระจาย ซึ่งไม่เป็นอันตรายแต่อย่างใดและไม่ได้บ่งชี้ว่าเกิดการลุกไหม้ใดๆ ในห้องโดยสาร', metadata={'source': 'docs/Text_PDF_.pdf', 'page': 75})

# CHAT

In [7]:
# Memory

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    output_key='answer',
)

In [8]:
model_lists = [ "gpt-3.5-turbo", 'gpt-4-1106-preview', "gpt-4", ]

llm = ChatOpenAI( api_key = api, model_name = model_lists[0], temperature = 0 )

In [34]:
# qa = RetrievalQA.from_chain_type(
#     llm,
#     retriever=db.as_retriever(search_type="similarity", search_kwargs={"k": 1}),
#     chain_type="map_reduce",
#     return_source_documents = True,
# )

In [31]:
memory.clear()

In [29]:
qa = ConversationalRetrievalChain.from_llm(
            llm = llm,
            retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 3}),
            memory = memory,
            return_source_documents = True,
            return_generated_question = True,
        )

## Get answer (Testing)

In [32]:
ran = splits[random.randint(0,len(splits))]

prompt = f"""
context : '{ran.page_content}'
Create 1 question using only the context provided.
End each question with a '?' character.
You must answer the question by refer to the context as much as you can.
Separate question/answer by a newline.
the question and the answer should be thai language.
"""

# Test
result = qa({"question": prompt})

print(f"context : {(ran.page_content)[:100]}\npage : {(ran.metadata)['page']+1}\n\n")

print(f"{result.keys()}\n\n")
print(f"Question :\n{result['generated_question']}\n\nAnswer :\n{result['answer']}\n\nSource :\nPage : {(result['source_documents'])[0].metadata}\nContent : {(result['source_documents'])[0].page_content}")

context : 2. ถอดสตอปเปอร์โลหะ (C) และถอดหลอดไฟ (D) 3. การติดตั้งหลอดไฟ ให้ทําย้อนขั้นตอนการถอด หมายเหตุ ●ติดตั
page : 351


dict_keys(['question', 'chat_history', 'answer', 'source_documents', 'generated_question'])


Question :

context : '2. ถอดสตอปเปอร์โลหะ (C) และถอดหลอดไฟ (D) 3. การติดตั้งหลอดไฟ ให้ทําย้อนขั้นตอนการถอด หมายเหตุ ●ติดตั้งฝาเบ้าหลอดไฟ (B) โดยให้ลูกศรชี้ไปทางด้านหน้าของรถ ไฟหรี่ (รถรุ่นที่ใช้ไฟหน้าแฮโลเจน) E01003702853 1. หมุนเบ้าหลอดไฟ (A) ทวนเข็มนาฬิกาเพื่อถอดออกแล้วจึงดึงหลอดไฟ (B) ออกจากเบ้า 2. การติดตั้งหลอดไฟ ให้ทําย้อนขั้นตอนการถอด ไฟเลี้ยวด้านหน้า E01003802883 รถรุ่นที่ใช้ไฟหน้าแฮโลเจน 1. ถอดคลิป (A) และจากนั้น ถอดฝาครอบด้านบน (B) 2. หมุนเบ้าหลอดไฟ (C) ทวนเข็มนาฬิกาเพื่อถอดออกแล้วจึงดึงหลอดไฟ (D) ออกจากเบ้า 3. การติดตั้งหลอดไฟ ให้ทําย้อนขั้นตอนการถอด หมายเหตุ ●เมื่อจะใส่คลิป (A) กลับเข้าไป ให้ใส่ส่วน (E)'
Create 1 question using only the context provided.
End each question with a '?' character.
You must answer the question by refer to the context as much as yo

In [33]:
((result['source_documents'])[0].metadata)['page']+1

351

In [14]:
# del result

# Prepare_dataset

In [34]:
import pandas as pd

df = pd.read_csv("~/Documents/DATAScience/TextAnalytics/Chatbot_LLM/rerank_dataset.csv", index_col=0)

df.tail(3)

,questions,answer,doc_page,doc_content,doc_source
6,คุณสามารถตั้งค่าหรือรีเซ็ตระยะทางในการเปลี่ยนย...,ใช่ คุณสามารถตั้งค่าหรือรีเซ็ตระยะทางในการเปลี...,101,เลือก “การบํารุงรักษา” โดยใช้แป้นเลื่อนแล้วกด ...,docs/Text_PDF_.pdf
7,การนั่งในท่าที่ถูกต้องมีความสำคัญอย่างไร?,การนั่งในท่าที่ถูกต้องมีความสำคัญอย่างมากเพื่อ...,76,- ถุงลมนิรภัยพองตัวอย่างรวดเร็วมาก ในบางกรณีอ...,docs/Text_PDF_.pdf
8,ควรทำอย่างไรเมื่อต้องเริ่มเครื่องยนต์ของรถ?,เมื่อต้องเริ่มเครื่องยนต์ของรถ คุณควรทำตามขั้น...,163,2. ตรวจให้แน่ใจว่าใส่เบรกมือแล้ว 3. เหยียบเบรก...,docs/Text_PDF_.pdf


In [35]:
rerank_df = {"questions":[], "answer":[], "doc_page":[], "doc_content":[], "doc_source":[]}

In [36]:
for i in range(51):
    ran = splits[random.randint(0,len(splits))]
    prompt = f"""
context : '{ran.page_content}'
Create 1 question using only the context provided.
End each question with a '?' character.
You must answer the question by refer to the context as much as you can.
Separate question/answer by a newline.
the question and the answer should be thai language.
"""
    try :
        result = qa({"question": prompt})
        
        rerank_df["answer"].append(result['answer'])
        rerank_df["questions"].append(result['generated_question'])
        rerank_df["doc_content"].append(ran.page_content)
        rerank_df["doc_page"].append( (ran.metadata)['page']+1 )
        rerank_df["doc_source"].append( (ran.metadata)['source'] )
        
    except :
        memory.clear()
        result = qa({"question": prompt})
 

In [37]:
r_df = pd.DataFrame.from_dict(rerank_df)

print(f"number of dataset : {len(r_df)}\n\n")
r_df.tail(3)

number of dataset : 42




,questions,answer,doc_page,doc_content,doc_source
39,ไฟแสดงการทำงานของโหมดการขับเคลื่อน 4 ล้อจะแสดง...,ไฟแสดงการทำงานของโหมดการขับเคลื่อน 4 ล้อจะแสดง...,206,: กะพริบ (ตัวแสดงโหมดการขับเคลื่อน4 ล้อกะพริบด...,docs/Text_PDF_.pdf
40,เมื่อขับรถอยู่บนทางลาดชันควรสตาร์ทเครื่องยนต์ใ...,"ควรสตาร์ทเครื่องยนต์ในตำแหน่ง ""P"" (จอด) เมื่อข...",184,คําเตือน ●ห้ามเลื่อนคันเกียร์ไปที่ตําแหน่ง “N”...,docs/Text_PDF_.pdf
41,ขนาดกระบอกสูบของรถคือเท่าไหร่?,ขนาดกระบอกสูบของรถคือ 86.0 มม.,355,จํานวนกระบอกสูบ : สูบเรียง 4 สูบ ปริมาตรกระบอก...,docs/Text_PDF_.pdf


In [38]:
df = pd.concat([df, r_df], 
                  ignore_index = True)

print(f"number of dataset : {len(df)}\n\n")
display(df)

number of dataset : 51




,questions,answer,doc_page,doc_content,doc_source
0,สวิตช์ล็อกทำงานอย่างไร?,เมื่อกดสวิตช์ล็อก (1) ไฟแสดง (2) จะสว่างขึ้น แ...,49,และดึงสวิตช์ขึ้นเพื่อปิดกุญแจอัจฉริยะ หมายเหตุ...,docs/Text_PDF_.pdf
1,การขับขี่ขึ้นทางชัน E00647000125 ต้องตั้งค่าปุ...,ไม่มีข้อมูลที่ระบุว่าต้องตั้งค่าปุ่มเลือกโหมดก...,215,ศูนย์บริการมิตซูบิชิจะไม่รับผิดชอบ วิธีการใช้ร...,docs/Text_PDF_.pdf
2,อุปกรณ์ภายนอกที่อยู่ด้านหน้าของรถมีอะไรบ้าง?,อุปกรณ์ภายนอกที่อยู่ด้านหน้าของรถประกอบด้วย:\n...,6,อุปกรณ์ภายนอก-ด้านหน้า ภาพโดยรวมคู่มืออย่างง่า...,docs/Text_PDF_.pdf
3,รถรุ่นที่ติดตั้งการเตือนคาดเข็มขัดนิรภัยสำหรับ...,"สัญลักษณ์บนเบาะนั่งสีแดงคือ ""X""",59,เป็นเหตุให้เสียงเตือนดังและไฟเตือนสว่างขึ้นได้...,docs/Text_PDF_.pdf
4,บนถนนที่ปกคลุมด้วยหิมะหรือลื่นอาจมีความแตกต่าง...,"ใช่, เส้นคาดการณ์แนวทางรถบนถนนที่ปกคลุมด้วยหิม...",265,เส้นคาดการณ์ แนวทางรถอาจแสดงไม่ถูกต้อง • บ...,docs/Text_PDF_.pdf
5,ควรเลือกโหมดการขับเคลื่อนใดเมื่อขับรถบนถนนลาดย...,ควรเลือกโหมดการขับเคลื่อน 4H (4-wheel drive hi...,194,: สว่าง : ดับ ตําแหน่งปุ่มเลือกการขับเคลื่อน 4...,docs/Text_PDF_.pdf
6,คุณสามารถตั้งค่าหรือรีเซ็ตระยะทางในการเปลี่ยนย...,ใช่ คุณสามารถตั้งค่าหรือรีเซ็ตระยะทางในการเปลี...,101,เลือก “การบํารุงรักษา” โดยใช้แป้นเลื่อนแล้วกด ...,docs/Text_PDF_.pdf
7,การนั่งในท่าที่ถูกต้องมีความสำคัญอย่างไร?,การนั่งในท่าที่ถูกต้องมีความสำคัญอย่างมากเพื่อ...,76,- ถุงลมนิรภัยพองตัวอย่างรวดเร็วมาก ในบางกรณีอ...,docs/Text_PDF_.pdf
8,ควรทำอย่างไรเมื่อต้องเริ่มเครื่องยนต์ของรถ?,เมื่อต้องเริ่มเครื่องยนต์ของรถ คุณควรทำตามขั้น...,163,2. ตรวจให้แน่ใจว่าใส่เบรกมือแล้ว 3. เหยียบเบรก...,docs/Text_PDF_.pdf
9,วิธีการเปิด/ปิดการตรวจจับวัตถุเคลื่อนที่ในระบบ...,วิธีการเปิด/ปิดการตรวจจับวัตถุเคลื่อนที่ในระบบ...,97,ที่แสดงใน เมนู “เซ็นเซอร์จอดรถ” ให้ใช้แป้นเลื่...,docs/Text_PDF_.pdf


In [39]:
df.to_csv("~/Documents/DATAScience/TextAnalytics/Chatbot_LLM/rerank_dataset.csv")